# Dynamic Sign language Recognition

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Keypoints using mpholistic

In [ ]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False          
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)      # its draw connection of all body landmarks
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

def draw_styled_landmarks(image, results):
  
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), # visual style for drawing, including color, thickness, and circle_radius.
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
  
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,121,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(90,80,40), thickness=2, circle_radius=2)
                             ) 
  
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,222,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(85,122,224), thickness=2, circle_radius=2)
                             ) 
   
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(66,245,236), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(235,245,96), thickness=2, circle_radius=2)
                             ) 

In [ ]:
cap= cv2.VideoCapture(0)
# url=r"http://192.0.0.4:8080/video"
# cap=cv2.VideoCapture(url)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
   while cap.isOpened():

     ret, frame=cap.read()
     image, results = mediapipe_detection(frame, holistic)
     print(results)
     
     draw_styled_landmarks(image, results)

     cv2.imshow('DSLR frame', image)

     key = cv2.waitKey(1)

     if key==20:
         break

cap.release()
cv2.destroyAllWindows()

In [ ]:
print(results)
len(results.face_landmarks.landmark)

**Keypoint Extraction**

In [ ]:
total_pose_lm=len(results.pose_landmarks.landmark)
print('\n', total_pose_lm, '\n')

px=results.pose_landmarks.landmark[0].x    # extrack landmark point from the pose landmarks.
py=results.pose_landmarks.landmark[0].y
pz=results.pose_landmarks.landmark[0].z
p_visibility=results.pose_landmarks.landmark[0].visibility
print(px,py,pz,p_visibility)

In [ ]:
pose=[]
for res in results.pose_landmarks.landmark:
    test=np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

print(test,('\n'))
len(pose)

In [ ]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
  

In [ ]:
lh
# rh
# pose
# face

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([pose, face, lh, rh])

shape_extact_keyponit=extract_keypoints(results).shape
print(shape_extact_keyponit)

test_result=extract_keypoints(results)
test_result


In [ ]:
np.save('0',test_result)

np.load('0.npy')

# Data Collection & Preprocessing

In [ ]:
Data_path = os.path.join("slr_Dataset")
actions = np.array(['Hello', 'I Love You', 'Sorry', 'Thank You'])
Sequences_no = 30
sequence_length = 30

In [ ]:
# for data accuracy here used saperate videos for each Numpy file to extract key points

video_path = r"C:\Users\salam_jmvmbso\OneDrive\Desktop\Dynamic_SLR\Videos\avi_Videos\thanks\thanks_57.avi"
cap = cv2.VideoCapture(video_path)

holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

for action in actions:
    for sequence in range(Sequences_no):
        try:
            os.makedirs(os.path.join(Data_path, action, str(sequence)))
        except:
            pass

        for frame_num in range(sequence_length):
            ret, frame = cap.read()

            if not ret:
                break

            image, results = mediapipe_detection(frame, holistic)

            draw_styled_landmarks(image, results)

            if frame_num == 0: 
                cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                cv2.imshow('DSLR Frame', image)
                cv2.waitKey(2000)
            else: 
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                cv2.imshow('DSLR Frame', image)

            keypoints = extract_keypoints(results)
            npy_path = os.path.join(Data_path, action, str(sequence), str(frame_num))
            np.save(npy_path, keypoints)

            key = cv2.waitKey(1)

            if key==30:
               break

cap.release()
cv2.destroyAllWindows()

# Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(Sequences_no):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(Data_path, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Model Training (LSTM)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
model = Sequential()   #intialize sequential model
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))   #add lstm layer to model, having 64 units
model.add(LSTM(128, return_sequences=True, activation='relu'))                         #returns sequence use the ectified linear unit (ReLU)
model.add(LSTM(64, return_sequences=False, activation='relu'))                         #i/p shape sequence 30 time step . each step having 1662 feature
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#cofigure learning process  Adam is optimization algo 

In [ ]:
log_dir = os.path.join('Logs1')

tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model.fit(X_train, y_train, epochs=2000,callbacks=[tb_callback])

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
model.summary()

# Prediction

In [ ]:
res = model.predict(X_test)
res

In [ ]:
actions[np.argmax(res[0])]

In [ ]:
actions[np.argmax(y_test[2])]

In [ ]:
model.save('DSLR_action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
from scipy import stats

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
# def prob_viz(res, actions, image, colors):
#     output_frame = image.copy()
#     for num, prob in enumerate(res):
#         cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
#         cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
#     return output_frame

In [ ]:
def prob_viz(res, actions, image, colors):
    output_frame = image.copy()
    for num, prob in enumerate(res):
        if num < len(actions) and num < len(colors):
            cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
            cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame


In [ ]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# url=r"http://192.0.0.4:8080/video"
# cap=cv2.VideoCapture(url)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
    
        draw_styled_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]   # makes prediction and lableing and apends both 
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])   #check probability and empty value
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('DSLR Frame', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()